In [ ]:
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
import os
from pinecone import Pinecone
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document
from langchain.retrievers import EnsembleRetriever

# 환경 변수 로드
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = os.getenv("PINECONE_INDEX_NAME")

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(pinecone_index_name)
embedding_model = OpenAIEmbeddings(api_key=openai_api_key)

vectorstore_orders = PineconeVectorStore(index=index, embedding=embedding_model, text_key="original_text", namespace="orders")
vectorstore_members = PineconeVectorStore(index=index, embedding=embedding_model, text_key="original_text", namespace="members")
# vectorstore_qna = PineconeVectorStore(index=index, embedding=embedding_model, text_key="original_text", namespace="qna")
 
retriever_orders = vectorstore_orders.as_retriever(search_kwargs={"k": 50})
retriever_members = vectorstore_members.as_retriever()
# retriever_qna = vectorstore_qna.as_retriever()

ensemble_retriever = EnsembleRetriever(retrievers=[retriever_orders, retriever_members])

# 프롬프트 템플릿 생성
template = """
당신은 BreadScanso라는 베이커리의 친절한 매출 분석가입니다. 
매장 A는 서초점, 매장 B는 강남점입니다. 
주어진 정보를 사용하여 다음 질문에 답하세요:

질문: {question}

이용 가능한 데이터:
{context}

지시사항:
1. 주어진 데이터에서 관련 정보를 모두 찾으세요.
2. 날짜별로 매출을 합산하고 분류하세요.
3. 데이터가 불완전하거나 없는 경우 명확히 언급하세요.
4. 수치 계산이 필요한 경우 단계별로 계산 과정을 보여주세요.

답변:
"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=["question", "context"]
)

llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=ensemble_retriever, chain_type_kwargs={"prompt": PROMPT})

def ask_question(question):
    try:
        response = qa_chain.invoke({"query": question})
        return response["result"] if "result" in response else response
    except Exception as e:
        print(f"오류 발생!!: {e}")
        return f"오류가 발생했습니다: {str(e)}"